## Manage Datasets in your GeoDB

The API server is hosted at:

- Hostname: https://3.120.53.215.nip.io/


### Connecting to the GeoDB

Your credentials will be configured already if you have purchased access to teh GeoDB from EOX. Otherwise a client Id and secret will have provided to you.

In [51]:
from dcfs_geodb.core.geodb import GeoDBClient

In [52]:
# add namespace=

geodb = GeoDBClient()
geodb.whoami

'geodb_admin'

In [53]:
# Use table instead of dataset
ds = geodb.get_collections()
ds


,table_name
0,land_use


In [54]:
if 'land_use' in ds['table_name'].values:
    geodb.drop_collection('land_use')

INFO:geodb.core:Dataset land_use deleted


In [55]:
ds = geodb.get_collections()
ds

,table_name


### Creating collections

Once the connection has bee4n established you will be able to create a table for datasets. The table will contain standard properties (fields). The lsit properties can be extended by the user.

In [56]:
# Have a look at fiona feature schema
collections = [
    {
        'name': 'land_use', 
        'crs': 3794,
        'properties': 
        [
            {'name': 'RABA_PID', 'type': 'float'}, 
            {'name': 'RABA_ID', 'type': 'float'}, 
            {'name': 'D_OD', 'type': 'date'}
        ]
    }
]

# return obj implementing __repr_html__
geodb.create_collections(collections)

INFO:geodb.core:Datasets {'collections': [{'name': 'land_use', 'crs': 3794, 'properties': [{'name': 'RABA_PID', 'type': 'float'}, {'name': 'RABA_ID', 'type': 'float'}, {'name': 'D_OD', 'type': 'date'}]}]} added.


True

## Loading data into a dataset

Once the table has been created, you can load data into the dataset. The example below loads a shapefile. The attributes of the shapefile correspond to the dataset's properties.


In [57]:
import geopandas
gdf = geopandas.read_file('data/sample/land_use.shp')
gdf

,RABA_PID,RABA_ID,D_OD,geometry
0,4770326.0,1410,2019-03-26,"POLYGON ((453952.629 91124.177, 453952.696 911..."
1,4770325.0,1300,2019-03-26,"POLYGON ((453810.376 91150.199, 453812.552 911..."
2,2305689.0,7000,2019-02-25,"POLYGON ((456099.635 97696.070, 456112.810 976..."
3,2305596.0,1100,2019-02-25,"POLYGON ((455929.405 97963.785, 455933.284 979..."
4,2310160.0,1100,2019-03-11,"POLYGON ((461561.512 96119.256, 461632.114 960..."
...,...,...,...,...
9822,6253989.0,1600,2019-03-08,"POLYGON ((460637.334 96865.891, 460647.927 969..."
9823,6252044.0,1600,2019-03-26,"POLYGON ((459467.868 96839.686, 459467.770 968..."
9824,6245985.0,2000,2019-04-08,"POLYGON ((459488.998 94066.248, 459498.145 940..."
9825,6245986.0,2000,2019-02-20,"POLYGON ((459676.680 94000.000, 459672.469 939..."


In [58]:
# insert shapefile
geodb.insert_into_collection('land_use', gdf)

INFO:geodb.core:Data inserted into land_use


True

In [59]:
# filter_table
geodb.filter_collection('land_use', query="raba_id=eq.7000")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,3,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((456099.635 97696.070, 456112.810 976...",2305689,7000,2019-02-25
1,26,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((459898.930 100306.841, 459906.288 10...",2301992,7000,2019-04-06
2,95,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((459591.248 92619.056, 459592.745 926...",2333229,7000,2019-02-20
3,115,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((459013.303 100354.458, 459022.756 10...",2336738,7000,2019-03-15
4,129,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((460851.200 93442.039, 460846.405 934...",6292562,7000,2019-02-21
...,...,...,...,...,...,...,...
379,9419,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((456523.670 94000.000, 456550.368 939...",6187993,7000,2019-03-13
380,9546,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((460793.279 94000.000, 460797.466 939...",6219403,7000,2019-02-21
381,9560,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((461750.000 98434.292, 461750.000 984...",6219602,7000,2019-03-20
382,9624,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((463628.912 97501.218, 463636.859 975...",6215237,7000,2019-03-20


In [60]:
geodb.delete_from_collection('land_use', query="raba_id=eq.7000")

INFO:geodb.core:Data from land_use deleted


True

In [61]:
geodb.filter_collection('land_use', query="raba_id=eq.7000")

,Empty Result


In [62]:
geodb.filter_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,2,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((453810.376 91150.199, 453812.552 911...",4770325,1300,2019-03-26
1,10,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2019-03-14
2,63,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((456201.531 98685.274, 456199.109 986...",2304287,1300,2019-02-25
3,86,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((454709.766 97354.278, 454704.878 973...",2331038,1300,2019-01-05
4,87,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((453820.737 98574.017, 453816.740 985...",2357574,1300,2019-01-16
...,...,...,...,...,...,...,...
890,9766,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2019-03-04
891,9767,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2019-04-01
892,9798,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2019-01-05
893,9799,2020-01-24T10:06:15.581907+00:00,None,"POLYGON ((461378.845 91124.318, 461490.224 911...",6264865,1300,2019-02-21


In [63]:
geodb.update_collection('land_use', query="raba_id=eq.1300", values={'d_od': '2000-01-01'})

INFO:geodb.core:land_use updated


True

In [64]:
geodb.filter_collection('land_use', query="raba_id=eq.1300")

,id,created_at,modified_at,geometry,raba_pid,raba_id,d_od
0,10,2020-01-24T10:06:15.581907+00:00,2020-01-24T10:07:48.07096+00:00,"POLYGON ((456547.427 91543.640, 456544.255 915...",2318555,1300,2000-01-01
1,103,2020-01-24T10:06:15.581907+00:00,2020-01-24T10:07:48.07096+00:00,"POLYGON ((456100.880 96973.323, 456111.084 969...",2332420,1300,2000-01-01
2,108,2020-01-24T10:06:15.581907+00:00,2020-01-24T10:07:48.07096+00:00,"POLYGON ((454070.991 90528.946, 454064.259 905...",4674624,1300,2000-01-01
3,117,2020-01-24T10:06:15.581907+00:00,2020-01-24T10:07:48.07096+00:00,"POLYGON ((454637.692 96679.827, 454630.584 966...",2336101,1300,2000-01-01
4,136,2020-01-24T10:06:15.581907+00:00,2020-01-24T10:07:48.07096+00:00,"POLYGON ((462781.112 100486.430, 462803.178 10...",2334416,1300,2000-01-01
...,...,...,...,...,...,...,...
890,9765,2020-01-24T10:06:15.581907+00:00,2020-01-24T10:07:48.07096+00:00,"POLYGON ((458304.555 100146.103, 458306.726 10...",6268058,1300,2000-01-01
891,9766,2020-01-24T10:06:15.581907+00:00,2020-01-24T10:07:48.07096+00:00,"POLYGON ((458247.583 99746.065, 458245.202 997...",6268059,1300,2000-01-01
892,9767,2020-01-24T10:06:15.581907+00:00,2020-01-24T10:07:48.07096+00:00,"POLYGON ((461932.283 90520.703, 461931.619 905...",6263561,1300,2000-01-01
893,9798,2020-01-24T10:06:15.581907+00:00,2020-01-24T10:07:48.07096+00:00,"POLYGON ((457231.170 90902.256, 457234.861 909...",6264862,1300,2000-01-01


## Managing Properties

In [40]:
geodb = GeoDBClient()
geodb.whoami

'geodb_admin'

In [65]:
geodb.get_collections()

,table_name
0,land_use


In [66]:
# collection
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date


In [67]:
geodb.add_property('land_use', "test_prop", 'integer')

INFO:geodb.core:Properties added


True

In [68]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
7,land_use,test_prop,integer


In [69]:
geodb.drop_property('land_use', 'test_prop')

INFO:geodb.core:Properties ['test_prop'] dropped from land_use


True

In [70]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date


In [71]:
geodb.add_properties('land_use', properties=[{'name': 'test1', 'type': 'integer'}, {'name': 'test2', 'type': 'date'}])

INFO:geodb.core:Properties added


True

In [72]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
7,land_use,test1,integer
8,land_use,test2,date


In [73]:
geodb.drop_properties('land_use', properties=['test1', 'test2'])

INFO:geodb.core:Properties ['test1', 'test2'] dropped from land_use


True

In [74]:
geodb.get_properties('land_use')

,table_name,column_name,data_type
0,land_use,id,integer
1,land_use,created_at,timestamp with time zone
2,land_use,modified_at,timestamp with time zone
3,land_use,geometry,USER-DEFINED
4,land_use,raba_pid,double precision
5,land_use,raba_id,double precision
6,land_use,d_od,date
